In [1]:
import pandas as pd
from datetime import date, datetime
from datetime import datetime as dt

# Fon Fiyatları Çekimi

In [2]:
concated2020 = pd.read_csv("concated_mainyear_2020.csv").reset_index(drop=True)
concated2020 = concated2020.iloc[:, 1:] 
concated2021 = pd.read_csv("concated_mainyear_2021.csv").reset_index(drop=True)
concated2021 = concated2021.iloc[:, 1:] 
concated2022 = pd.read_csv("concated_mainyear_2022.csv").reset_index(drop=True)
concated2022 = concated2022.iloc[:, 1:] 
concated2023 = pd.read_csv("concated_mainyear_2023.csv").reset_index(drop=True)
concated2023 = concated2023.iloc[:, 1:] 
sureklicekimfonlar = pd.read_csv("sureklicekimfonlar.csv").reset_index(drop=True)
sureklicekimfonlar = sureklicekimfonlar.iloc[:, 1:] 
df = pd.concat([concated2020,
               concated2021,
               concated2022,
               concated2023,
               sureklicekimfonlar],
              ignore_index=True)
df

,Tarih,Fon Kodu,Fon Adı,Fiyat,Tedavüldeki Pay Sayısı,Kişi Sayısı,Fon Toplam Değer
0,02.01.2020,AAK,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,35.677757,"456,001.0",303,"16,269,093.1"
1,02.01.2020,AAL,ATA PORTFÖY PARA PİYASASI (TL) FONU,0.575406,"201,742,677.0","3,046","116,083,937.71"
2,02.01.2020,AAS,ATA PORTFÖY FON SEPETİ SERBEST FONU,0.025357,"128,719,961.0",7,"3,263,938.64"
3,02.01.2020,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.031263,"2,868,807,200.0",11,"89,688,581.9"
4,02.01.2020,ABU,AK PORTFÖY BİREBİR BANKACILIK DEĞİŞKEN ÖZEL FON,0.019345,"612,106,546.0",117,"11,841,232.29"
...,...,...,...,...,...,...,...
877627,22.01.2024,ZVB,ATLAS PORTFÖY SERBEST (TL) FON,26.135242,"3,750,538.0",10,"98,021,218.21"
877628,22.01.2024,ZVO,ZİRAAT PORTFÖY ÜÇÜNCÜ SERBEST (TL) FON,2.616363,"222,059,759.0",12,"580,988,926.05"
877629,22.01.2024,ZYC,ZİRAAT PORTFÖY DÖRDÜNCÜ SERBEST (TL) FON,1.606412,"1,833,786,550.0",1,"2,945,816,603.1"
877630,22.01.2024,ZYD,ZİRAAT PORTFÖY BEŞİNCİ SERBEST (TL) FON,1.607651,"1,844,187,892.0",1,"2,964,810,051.78"


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from io import StringIO
from selenium.webdriver.support.ui import Select

while True:
    driver = webdriver.Chrome()

    start_date = pd.to_datetime('today')
    end_date = pd.to_datetime('today')

    driver.get("https://www.tefas.gov.tr/TarihselVeriler.aspx")
    
    time.sleep(10)
    
    # Set start date
    driver.execute_script('document.getElementById("TextBoxStartDate").setAttribute("value", arguments[0])', start_date.strftime('%d.%m.%Y'))

    # Set end date
    driver.execute_script('document.getElementById("TextBoxEndDate").setAttribute("value", arguments[0])', end_date.strftime('%d.%m.%Y'))

    # Click the search button
    driver.find_element(By.ID, "ButtonSearchDates").click()

    print("Yükleniyor...")

    # Wait for the data to load (adjust the time.sleep value if needed)
    time.sleep(60)

    # Check if the table data is present
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "td.dt-center.sorting_1")))
        print("Veriler girilmiş.")

        print("Sayfa yapılandırılıyor...")

        dropdown = driver.find_element(By.NAME, "table_general_info_length")
        select = Select(dropdown)
        select.select_by_value("250")

        print("Sayfa / yeni sayfa çekimine başlandı...")
        df_list = []
        while True:
            # Sayfa kaynağını çek
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            table = soup.find('table', {'id': 'table_general_info'})
            html_content = str(table)
            df = pd.read_html(StringIO(html_content), thousands='.')[0]
            df["Tarih"] = pd.read_html(StringIO(html_content))[0]["Tarih"]
            # Elde edilen veriyi listeye ekle
            df_list.append(df)

            # "Sonraki" butonunun durumunu kontrol et
            next_button = driver.find_element(By.ID, "table_general_info_next")
            if "disabled" in next_button.get_attribute("class"):
                break  # Eğer buton etkisizse döngüyü kır

            # "Sonraki" butonuna tıkla ve bekle
            next_button.click()
            time.sleep(1.3)

        print("Sayfa / yeni sayfa çekimi tamamlandı...")
        print("Birleştiriliyor...")
        df = pd.concat(df_list, ignore_index=True)
        driver.quit()
        print(start_date + end_date + " çekimi tamamlandı.")

        print("Formatlanıyor...")
        newdf = df.copy()
        newdf['Tarih'] = pd.to_datetime(df['Tarih'], format="%d.%m.%Y")
        newdf['Tarih'] = newdf['Tarih'].dt.strftime("%d.%m.%Y")
        newdf['Fon Kodu'] = df['Fon Kodu'].astype(str)
        newdf['Fon Adı'] = df['Fon Adı'].astype(str)

        newdf['Fiyat'] = pd.to_numeric(df['Fiyat'].str.replace(".", "").str.replace(",", "."))
        newdf['Fiyat'] = newdf.apply(lambda x: "{:,}".format(x['Fiyat']), axis=1)

        newdf['Tedavüldeki Pay Sayısı'] = pd.to_numeric(df['Tedavüldeki Pay Sayısı'].str.replace(".", "").str.replace(",", "."))
        newdf['Tedavüldeki Pay Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Tedavüldeki Pay Sayısı']), axis=1)

        newdf['Fon Toplam Değer'] = pd.to_numeric(df['Fon Toplam Değer'].str.replace(".", "").str.replace(",", "."))
        newdf['Fon Toplam Değer'] = newdf.apply(lambda x: "{:,}".format(x['Fon Toplam Değer']), axis=1)

        newdf['Kişi Sayısı'] = pd.to_numeric(df['Kişi Sayısı'])
        newdf['Kişi Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Kişi Sayısı']), axis=1)

        q1_2023 = newdf.copy()
        print("Formatlama tamamlandı.")
        print("q1_2023 olarak kullanımda.")

        break  # Break the loop if the operation is successful

    except:
        print("Veriler girilmemiş, 30 saniye sonra yeniden denenecek.")
        # Continue the loop for another attempt
        continue

    finally:
        driver.quit()
        time.sleep(30)
        print("Deneme başlıyor...")

Yükleniyor...
Veriler girilmemiş, 30 saniye sonra yeniden denenecek.
Deneme başlıyor...


KeyboardInterrupt: 